#[Your_name]

In [ ]:
# !pip install arabert
# !pip install camel-tools
# !camel_data -i light
# !pip install transformers
# !pip uninstall camel-tools
# !pip uninstall arabert

<h1 style="text-align: center;">Sequence Labeling<h1>

Prepare libraries

In [ ]:
# here put every import you need e.g. import nltk
# it's better to load what you need from the package by from [] import [] instead of import the whole package
import re
import nltk
from nltk import pos_tag, word_tokenize
from nltk.chunk import ne_chunk
import pandas as pd
from tokenization import tokenize as tk
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from camel_tools.tokenizers.word import simple_word_tokenize
from camel_tools.tagger.default import DefaultTagger
from sklearn.linear_model import LogisticRegression
# from camel_tools.utils.dediac import dediac_arabic_text
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from nltk.tag import UnigramTagger, BigramTagger
import stanza
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from tokenization import tokenize
from sklearn import metrics

Download data

In [ ]:
# !wget 'https://drive.google.com/uc?export=download&id=1-3JRPU4aq9t_RRujrJ1HpmAPUbRe0QYK' -O 'IOB.csv'

<h2 dir="rtl">مثال عن كيفية كتابة حلول الطلبات:</h2>


<div dir="rtl">شرح ما يقوم به الكود (like code documentation)<div>

In [ ]:
# your code here


In [ ]:
# example test

<div dir="rtl">ملاحظاتك في حال وجودها</div>
<div dir="rtl">يمكنك إضافة خلايا لكل طلب بقدر ما تشاء، المهم أن تحافظ على تنظيم الملف</div>

# Question [1]: Data Preparation

## [1.1]

In [ ]:
path = 'IOB.csv'

def read_file(path,file):
    schedule = pd.read_csv(path, names=['word','entity tag','POS','stopword'],sep=',',skiprows=1)
    schedule['file'] = file
    return schedule


schedual = read_file(path,'IOB')
schedual.head(5)

In [ ]:
entity = set(schedual['entity tag'].values)
entity

In [ ]:
def lines_to_tuples(words,tags):
   
    entities = []
    for word,tag in zip(words,tags):
        my_tuple = (word,tag)
        entities.append(my_tuple)

    return entities

# مثال على استخدام الدالة
result = lines_to_tuples(schedual['word'].values,schedual['entity tag'].values)
print(result)

## [1.2]

In [ ]:
phrases = []
def lines_to_phrases(words,tags,poss,stopwords):
    for word,tag,pos,stopword in zip(words,tags,poss,stopwords):
        phrase = ' '.join([word,tag,pos,stopword])
        phrases.append(phrase)


    return phrases

result = lines_to_phrases(schedual['word'].values,schedual['entity tag'].values,schedual['POS'].values,schedual['stopword'].values)
print(result)


## [1.3]

In [ ]:
list_of_lists = []
def lines_to_list_of_lists(words,tags,poss,stopwords):
    my_tuples  = lines_to_tuples(words,tags)
    my_phrases = lines_to_phrases(words,tags,poss,stopwords)

    for my_tuple,my_phrase in zip(my_tuples,my_phrases):
        list_of_lists.append([my_tuple,my_phrase])

    return list_of_lists    


lines_to_list_of_lists(schedual['word'].values,schedual['entity tag'].values,schedual['POS'].values,schedual['stopword'].values)

## [1.4]

In [ ]:
words_list = []
tags_list = []

def two_lists_word_and_tag(words,tags):
    for word,tag in zip(words,tags):
        words_list.append(word)
        tags_list.append(tag)

    return words_list,tags_list 


my_word_list,my_tag_list = two_lists_word_and_tag(schedual['word'].values,schedual['entity tag'].values)
print(my_word_list,my_tag_list)

## [1.5]

In [ ]:
from sklearn.model_selection import train_test_split

X = schedual[['word', 'POS', 'stopword']]
y = schedual['entity tag']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1000)



## [1.6]

In [ ]:
# عدد الجمل في كل من الترين والتيست

train_data_phrases_len = len(X_train['word'])
test_data_phrases_len = len(X_test['word'])
print(f'Train phrases length: {train_data_phrases_len}')
print(f'Test phrases length: {test_data_phrases_len}')

## [1.7]

In [ ]:
def calculate_named_entities_ratio(x_train_data,y_train_data):
    words_len = len(x_train_data['word'])
    tags_len = 0
    for i in y_train_data:
        if i == 'I' or i == 'B':
            tags_len +=1
    
    return tags_len/words_len
    
    
    
ratio = calculate_named_entities_ratio(X_train,y_train)
print(ratio)

## [1.8]

In [ ]:
from farasa.pos import FarasaPOSTagger
from farasa.segmenter import FarasaSegmenter

arabic_text = "Tony botros is the best in the whole world"

# Initialize the Farasa Segmenter in Standalone mode
farasa_segmenter = FarasaSegmenter(interactive=False)

# Tokenize the Arabic text
arabic_words = farasa_segmenter.segment(arabic_text)

# Initialize the Farasa POS tagger in Standalone mode
farasa_pos_tagger = FarasaPOSTagger(interactive=False)

# Perform part-of-speech tagging
pos_tags = farasa_pos_tagger.tag(arabic_words)

print(pos_tags)


## [1.9]

In [ ]:
count_per_category = y_train.value_counts()

most_common_category = count_per_category.idxmax()
print(f"الصنف الأكثر تكرارًا هو: {most_common_category}")


# Question [2]: Statistics

In [ ]:
sentence = "التصلب المتعدد مرض يحتمل أن يسبب إعاقة الدماغ و الحبل النخاعي (الجهاز العصبي المركزي). عند الإصابة بمرض التصلب المتعدد، يهاجم الجهاز المناعي غمد الحماية (المايلين) الذي يغطي الألياف العصبية، ويسبب مشكلات في الاتصال بين الدماغ وبقية الجسم. في النهاية، يمكن أن يسبب تلفاً أو تدهوراً دائما في الألياف العصبية.  "

In [ ]:
# sentence_tokenize=tk(sentence)
sentence_tokenize=tokenize(sentence)

In [ ]:
comparison_table = {}

comparison_table['question_step_number'] = []
comparison_table['model_name'] = []
comparison_table['features'] = []
comparison_table['accuracy'] = []

## [2.1]


In [ ]:
print(len(schedual[schedual['entity tag']=='O']))

In [ ]:
print(len(schedual))

In [ ]:
def taget_labet_classifier(words):
    return [(word,'O') for word in words]

In [ ]:
classified_text = taget_labet_classifier(sentence_tokenize)
# classified_text

In [ ]:
def calculate_accuracy(predicted_labels, true_labels):
    correct = sum(pred == true for pred, true in zip(predicted_labels, true_labels))
    accuracy = correct / len(predicted_labels) if predicted_labels else 0
    return accuracy

In [ ]:
predicted_labels = taget_labet_classifier(schedual['word'])

true_labels = schedual['entity tag'] 

true_labels_pairs = [(word, tag) for word, tag in zip(schedual['word'], true_labels)]

accuracy = calculate_accuracy(predicted_labels, true_labels_pairs)
print(f"accuracy : {accuracy}")

## [2.2]

In [ ]:
from nltk import FreqDist

# word_tokens = word_tokenize(' '.join(X_train['word'].values))
word_tokens = tk(' '.join(X_train['word'].values))
freq_dist = FreqDist(word_tokens)
top_100_words = freq_dist.most_common(100) 
# print(top_100_words)
top_100_words_only = [word for word, _ in top_100_words]
# print(top_100_words_only)


In [ ]:
filtered_schedual = schedual[schedual['word'].isin(top_100_words_only)].drop_duplicates().dropna()

In [ ]:
tagger_lookup = dict(zip(filtered_schedual['word'], filtered_schedual['entity tag']))

def classify_words(words):
    return [(word, tagger_lookup.get(word,'')) for word in words]

In [ ]:
classified_text = classify_words(sentence_tokenize)
# classified_text

In [ ]:
predicted_labels = classify_words(schedual['word'])

true_labels = schedual['entity tag'] 

true_labels_pairs = [(word, tag) for word, tag in zip(schedual['word'], true_labels)]

accuracy = calculate_accuracy(predicted_labels, true_labels_pairs)
print(f"accuracy : {accuracy}")

In [ ]:
default_tag = 'O'
def classify_with_backoff(words):
    return [(word, tagger_lookup.get(word, default_tag)) for word in words]

In [ ]:
classified_text = classify_with_backoff(sentence_tokenize)
# classified_text

In [ ]:
predicted_labels = classify_with_backoff(schedual['word'])

true_labels = schedual['entity tag'] 

true_labels_pairs = [(word, tag) for word, tag in zip(schedual['word'], true_labels)]

accuracy = calculate_accuracy(predicted_labels, true_labels_pairs)
print(f"accuracy : {accuracy}")

## [2.3]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(schedual['word'], schedual['entity tag'], test_size=0.2, random_state=1000)
train_data=dict(zip(X_train, y_train))
test_data=dict(zip(X_test,y_test))
tagger_list = [(word, tag) for word, tag in train_data.items()]
train_data = [[pair] for pair in tagger_list]
tagger_list = [(word, tag) for word, tag in test_data.items()]
test_data = [[pair] for pair in tagger_list]

In [ ]:
unigram_tagger = UnigramTagger(train_data)

unigram_accuracy = unigram_tagger.accuracy(test_data)
print(f"unigram_accuracy : {unigram_accuracy}")

In [ ]:
bigram_tagger = BigramTagger(train_data, backoff=unigram_tagger)

bigram_accuracy = bigram_tagger.accuracy(test_data)
print(f"bigram_accuracy : {bigram_accuracy}")

# Question [3]: ML Classification

## [3.1]


### [3.1.1]

In [ ]:
# stanza.download('ar')

nlp = stanza.Pipeline('ar', processors='tokenize,pos')
def extract_features(sentence, index):
    # إعداد محلل اللغة العربية

    # تحليل الجملة
    doc = nlp(sentence)

    features = {}

    # سمات صرفية للكلمة الحالية
    current_word = doc.sentences[0].words[index - 1].text
    features['current_word'] = current_word
    current_word_pos = doc.sentences[0].words[index - 1].upos
    # features['current_word_pos'] = doc.sentences[0].words[index - 1].upos
    # نوع الضمير إذا كانت الكلمة ضميراً
    if current_word_pos in ['PRON']:
        features['pronoun_type'] = 'subject' if current_word in ['أنا', 'أنت', 'هو', 'هي', 'نحن', 'أنتم', 'هم'] else 'object'
    else:
        features['pronoun_type'] = 'none'
    # المضاف إليه إذا كانت الكلمة اسم إضافة
    if current_word_pos in ['ADJ']:
        features['adjective'] = 'True'
    else:
        features['adjective'] = 'False'   

    # الجنس والعدد إذا كانت الكلمة اسماً
    if current_word_pos in ['NOUN', 'PRON']:
        # يمكنك تعديل الشروط والكلمات حسب اللغة العربية
        features['gender'] = 'female' if current_word.endswith("ة") else 'male'
        features['number'] = 'Feminine Plural' if current_word.endswith("ات") or current_word.endswith("يات") else 'Masculine Plural' if current_word.endswith("ون") or current_word.endswith("ين") else 'Singular'
    else:
        features['gender'] = 'none'
        features['number'] = 'none'


    # # سمات صرفية للكلمة السابقة والتالية
    # if index > 1:
    #     features['previous_word'] = doc.sentences[0].words[index - 2].text
    #     features['previous_word_pos'] = doc.sentences[0].words[index - 2].upos

    # if index < len(doc.sentences[0].words):
    #     features['next_word'] = doc.sentences[0].words[index].text
    #     features['next_word_pos'] = doc.sentences[0].words[index].upos

    return features, doc, current_word_pos


index = 2  
word_features,_,_ = extract_features(sentence, index)
print(word_features)


### [3.1.2]

In [ ]:
def features_and_the_word_before_and_after(sentence, index):
    features,doc,_ = extract_features(sentence, index)

    # سمات صرفية للكلمة السابقة والتالية
    if index > 1:
        features['previous_word'] = doc.sentences[0].words[index - 2].text
        # features['previous_word_pos'] = doc.sentences[0].words[index - 2].upos

    if index < len(doc.sentences[0].words):
        features['next_word'] = doc.sentences[0].words[index].text
        # features['next_word_pos'] = doc.sentences[0].words[index].upos

    return features

index = 10
print(features_and_the_word_before_and_after(sentence,index))
    

### [3.1.3]

In [ ]:
def features_and_pos(sentence,index):
    features,_,pos = extract_features(sentence,index)

    features['current_word_pos'] = pos

    return features

index = 10
print(features_and_pos(sentence,index))

### [3.1.4]

In [ ]:
def features_and_the_pos_for_word_before_and_after(sentence, index):
    features,doc,_ = extract_features(sentence, index)

    # سمات صرفية للكلمة السابقة والتالية
    if index > 1:
        # features['previous_word'] = doc.sentences[0].words[index - 2].text
        features['previous_word_pos'] = doc.sentences[0].words[index - 2].upos

    if index < len(doc.sentences[0].words):
        # features['next_word'] = doc.sentences[0].words[index].text
        features['next_word_pos'] = doc.sentences[0].words[index].upos

    return features

index = 10
print(features_and_the_pos_for_word_before_and_after(sentence,index))
    

### [3.1.5]

In [ ]:
def features_bigram(sentence,index):
    features,doc,pos = extract_features(sentence, index)

    # Previous word features
    if index > 0:
        prev_word = doc.sentences[0].words[index - 2].text
        prev_word_pos = doc.sentences[0].words[index - 2].upos
        features['prev_word'] = prev_word
        features['prev_word_pos'] = prev_word_pos

        # Bigram features for POS
        features['pos_bigram_prev_current'] = f"{prev_word_pos}_{pos}"

    # Next word features
    if index < len(doc.sentences[0].words) - 1:
        next_word = doc.sentences[0].words[index].text
        next_word_pos = doc.sentences[0].words[index].upos
        features['next_word'] = next_word
        features['next_word_pos'] = next_word_pos

        # Bigram features for POS
        features['pos_bigram_current_next'] = f"{pos}_{next_word_pos}"


    return features    


index = 10
print(features_bigram(sentence,index))

    

### [3.1.6]

In [ ]:
def features_and_prev_pos(sentence,index):
    features,doc,_ = extract_features(sentence, index)
    if index > 1:
        features['previous_word_pos'] = doc.sentences[0].words[index - 2].upos

    return features


index = 10
print(features_and_prev_pos(sentence,index))


### [3.1.7]

In [ ]:
def features_and_new_features(sentence,index):
    features,doc,_ = extract_features(sentence, index)

    current_word_form =  doc.sentences[0].words[index - 1].feats

    features['current_word_form'] = current_word_form

    return features


index = 10
print(features_and_new_features(sentence,index))

# """ 
# index 10 = 'الحبل'
# "Case=Gen": الكلمة في حالة الجر (Genitive).
# "Definite=Def": الكلمة محددة (Definite).
# "Number=Sing": الكلمة في صيغة المفرد (Singular).
# """


## [3.2]

In [36]:
def bayes_naive_multinomial(sentence, method,features_type,word_pos=False):
    tokenized_sentence = tokenize(sentence)
    all_Words = []
    words_pos = []
    # Extract features for all words in the sentence
    for i in range(len(tokenized_sentence)):
        features, _, pos = method(sentence, i)
        all_Words.append(features)
        if word_pos == True:
            words_pos.append(pos)


    if word_pos == True:
        y_all = [item for item in words_pos]
        X =  [item['current_word'] for item in all_Words]
    
        # Split the data into training and testing sets
        X_train, X_test, y_train, y_test = train_test_split(X, y_all, test_size=0.25, random_state=42)

        # Extract features and labels for training set
        X_train_features = [item for item in X_train]
        X_test_features = [item for item in X_test]

        # Create a CountVectorizer and transform the training and testing features
        vectorizer = CountVectorizer()
        X_train_vec = vectorizer.fit_transform(X_train_features)
        X_test_vec = vectorizer.transform(X_test_features)

        # Create and train the Naive Bayes classifier
        nb_classifier = MultinomialNB()
        nb_classifier.fit(X_train_vec, y_train)

        # Make predictions on the testing set
        y_pred = nb_classifier.predict(X_test_vec)

        # Evaluate the performance of the classifier
        accuracy = metrics.accuracy_score(y_test, y_pred)
        print(f"Accuracy: {accuracy:.2f}")


    else:
    # Extract the labels (pronoun_type) from the features
        y_all = [item[features_type] for item in all_Words]
        X =  [item['current_word'] for item in all_Words]

        
        # Split the data into training and testing sets
        X_train, X_test, y_train, y_test = train_test_split(X, y_all, test_size=0.25, random_state=42)

        # Extract features and labels for training set
        X_train_features = [item for item in X_train]
        X_test_features = [item for item in X_test]

        # Create a CountVectorizer and transform the training and testing features
        vectorizer = CountVectorizer()
        X_train_vec = vectorizer.fit_transform(X_train_features)
        X_test_vec = vectorizer.transform(X_test_features)

        # Create and train the Naive Bayes classifier
        nb_classifier = MultinomialNB()
        nb_classifier.fit(X_train_vec, y_train)

        # Make predictions on the testing set
        y_pred = nb_classifier.predict(X_test_vec)

        # Evaluate the performance of the classifier
        accuracy = metrics.accuracy_score(y_test, y_pred)
        print(f"Accuracy: {accuracy:.2f}")

# Example usage
bayes_naive_multinomial(sentence, extract_features,'pronoun_type')


Accuracy: 1.00


In [33]:
# pos
bayes_naive_multinomial(sentence, extract_features,'pos',word_pos=True)


Accuracy: 0.40


In [37]:
# adjective
bayes_naive_multinomial(sentence, extract_features,'adjective')


Accuracy: 0.93


In [38]:
# gender
bayes_naive_multinomial(sentence, extract_features,'gender')


Accuracy: 0.80


In [39]:
# number
bayes_naive_multinomial(sentence, extract_features,'number')
 


Accuracy: 0.80


## [3.3]

In [40]:
def multinomial_logistic_regression(sentence, method,features_type,word_pos=False):
    tokenized_sentence = tokenize(sentence)
    all_Words = []
    words_pos = []
    # Extract features for all words in the sentence
    for i in range(len(tokenized_sentence)):
        features, _, pos = method(sentence, i)
        all_Words.append(features)
        if word_pos:
            words_pos.append(pos)


    if words_pos:
        # Extract the labels (pronoun_type) from the features
        y_all = [item for item in words_pos]
        X =  [item['current_word'] for item in all_Words]

        # Split the data into training and testing sets
        X_train, X_test, y_train, y_test = train_test_split(X, y_all, test_size=0.25, random_state=42)

        X_train = [doc for doc in X_train if doc]
        X_test = [doc for doc in X_test if doc]
        
        # Extract features and labels for training set
        X_train_features = [item for item in X_train]
        X_test_features = [item for item in X_test]


        # Create a CountVectorizer and transform the training and testing features
        vectorizer = CountVectorizer()
        X_train_vec = vectorizer.fit_transform(X_train_features)
        X_test_vec = vectorizer.transform(X_test_features)

        # Create and train the Naive Bayes classifier
        logistic_classifier = LogisticRegression(multi_class='multinomial', solver='lbfgs')
        logistic_classifier.fit(X_train_vec, y_train)


        # Make predictions on the testing set
        y_pred = logistic_classifier.predict(X_test_vec)

        # Evaluate the performance of the classifier
        accuracy = metrics.accuracy_score(y_test, y_pred)
        print(f"Accuracy: {accuracy:.2f}")        

    else:
        # Extract the labels (pronoun_type) from the features
        y_all = [item[features_type] for item in all_Words]
        X =  [item['current_word'] for item in all_Words]

        # Split the data into training and testing sets
        X_train, X_test, y_train, y_test = train_test_split(X, y_all, test_size=0.25, random_state=42)

        X_train = [doc for doc in X_train if doc]
        X_test = [doc for doc in X_test if doc]
        
        # Extract features and labels for training set
        X_train_features = [item for item in X_train]
        X_test_features = [item for item in X_test]


        # Create a CountVectorizer and transform the training and testing features
        vectorizer = CountVectorizer()
        X_train_vec = vectorizer.fit_transform(X_train_features)
        X_test_vec = vectorizer.transform(X_test_features)


        # Create and train the Naive Bayes classifier
        logistic_classifier = LogisticRegression(multi_class='multinomial', solver='lbfgs')
        logistic_classifier.fit(X_train_vec, y_train)


        # Make predictions on the testing set
        y_pred = logistic_classifier.predict(X_test_vec)


        # Evaluate the performance of the classifier
        accuracy = metrics.accuracy_score(y_test, y_pred)
        print(f"Accuracy: {accuracy:.2f}")

# Example usage
multinomial_logistic_regression(sentence, extract_features,'adjective')


Accuracy: 0.93


In [41]:
# pos
multinomial_logistic_regression(sentence, extract_features,'pos',word_pos=True)


Accuracy: 0.40


In [42]:
# gender
multinomial_logistic_regression(sentence, extract_features,'gender')


Accuracy: 0.67


In [43]:
# number
bayes_naive_multinomial(sentence, extract_features,'number')
 

Accuracy: 0.80


## [3.4]

In [ ]:
from sklearn_crfsuite import CRF

def crf_model(sentence, method,features_type,word_pos=False):
    tokenized_sentence = tokenize(sentence)
    all_Words = []
    words_pos = []
    # Extract features for all words in the sentence
    for i in range(len(tokenized_sentence)):
        features, _, pos = method(sentence, i)
        all_Words.append(features)
        if word_pos:
            words_pos.append(pos)


    if words_pos:
        y_all = [item for item in words_pos]
        X = [{'word': item['current_word']} for item in all_Words]
        
        X_train, X_test, y_train, y_test = train_test_split(X, y_all, test_size=0.25, random_state=42)

        # X_train = [doc for doc in X_train if doc]
        # X_test = [doc for doc in X_test if doc]
        
        # # Extract features and labels for training set
        # X_train_features = [item for item in X_train]
        # X_test_features = [item for item in X_test]

        # X_train_crf = [dict(zip(item, item)) for item in X_train]
        # X_test_crf = [dict(zip(item, item)) for item in X_test]

        print(X_train)
        print(y_train)

        crf = CRF()
    
        try:
            crf.fit(X_train, y_train)
        except Exception as e:
            print(f"Error during model training: {e}")
            return


        # Make predictions on the testing set
        y_pred = crf.predict(X_train)
        print(len(X_train))
        print(len(y_test))
        print(len(y_pred))

        # Evaluate the performance of the classifier
        accuracy = metrics.accuracy_score(y_test, y_pred)
        print(f"Accuracy: {accuracy:.2f}")        

    else:
        # Extract the labels (pronoun_type) from the features
        y_all = [item[features_type] for item in all_Words]
        
        # Split the data into training and testing sets
        X_train, X_test, y_train, y_test = train_test_split(all_Words, y_all, test_size=0.25, random_state=42)

        X_train = [doc for doc in X_train if doc]
        X_test = [doc for doc in X_test if doc]
        
        # Extract features and labels for training set
        X_train_features = [item[features_type] for item in X_train]
        X_test_features = [item[features_type] for item in X_test]


        # Create a CountVectorizer and transform the training and testing features
        vectorizer = CountVectorizer()
        X_train_vec = vectorizer.fit_transform(X_train_features)
        X_test_vec = vectorizer.transform(X_test_features)


        # Create and train the Naive Bayes classifier
        logistic_classifier = LogisticRegression(multi_class='multinomial', solver='lbfgs')
        logistic_classifier.fit(X_train_vec, y_train)


        # Make predictions on the testing set
        y_pred = logistic_classifier.predict(X_test_vec)

        # Evaluate the performance of the classifier
        accuracy = metrics.accuracy_score(y_test, y_pred)
        print(f"Accuracy: {accuracy:.2f}")

# Example usage
crf_model(sentence, extract_features,'pos',word_pos=True)


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn_crfsuite import CRF
from sklearn.metrics import classification_report

# Sample data for part-of-speech tagging
data = [
    {'word': 'The', 'pos': 'DET'},
    {'word': 'quick', 'pos': 'ADJ'},
    {'word': 'brown', 'pos': 'ADJ'},
    {'word': 'fox', 'pos': 'NOUN'},
    {'word': 'jumps', 'pos': 'VERB'},
    {'word': 'over', 'pos': 'ADP'},
    {'word': 'the', 'pos': 'DET'},
    {'word': 'lazy', 'pos': 'ADJ'},
    {'word': 'dog', 'pos': 'NOUN'},
    {'word': '.', 'pos': 'PUNCT'},
]

# Modified feature extraction to include previous and next words
def word_features(sentence, i):
    features = {
        'word': sentence[i]['word'],
        'prev_word': '' if i == 0 else sentence[i - 1]['word'],
        'next_word': '' if i == len(sentence) - 1 else sentence[i + 1]['word'],
    }
    return features

# Extract features and labels
X = [word_features(data, i) for i in range(len(data))]
y = [item['pos'] for item in data]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Displaying the example data
print("Example X_train:")
for x in X_train:
    print(x)

print("\nExample y_train:")
print(y_train)

# Create and train the CRF model
crf = CRF()
crf.fit(X_train, y_train)

# Make predictions on the testing set
y_pred = crf.predict(X_test)

# Evaluate the performance of the classifier
report = classification_report(y_test, y_pred)
print("Classification Report:\n", report)


## [3.5]

In [49]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from hmmlearn import hmm
from sklearn import metrics

def my_hmm(sentence, method, features_type, word_pos=False):
    tokenized_sentence = tokenize(sentence)
    all_Words = []
    words_pos = []

    # Extract features for all words in the sentence
    for i in range(len(tokenized_sentence)):
        features, _, pos = method(sentence, i)
        all_Words.append(features)
        if word_pos:
            words_pos.append(pos)

    if words_pos:
        # Extract the labels (pronoun_type) from the features
        y_all = [item for item in words_pos]
        X = [item['current_word'] for item in all_Words]

        # Split the data into training and testing sets
        X_train, X_test, y_train, y_test = train_test_split(X, y_all, test_size=0.25, random_state=42)

        X_train = [doc for doc in X_train if doc]
        X_test = [doc for doc in X_test if doc]

        # Extract features and labels for the training set
        X_train_features = [item for item in X_train]
        X_test_features = [item for item in X_test]

        # Create a CountVectorizer and transform the training and testing features
        vectorizer = CountVectorizer()
        X_train_vec = vectorizer.fit_transform(X_train_features).toarray()
        X_test_vec = vectorizer.transform(X_test_features).toarray()

        # Now, let's create an HMM model
        # Assuming X_train_vec and X_test_vec are the features for the HMM model
        my_hmm_model = hmm.MultinomialHMM(n_components=3, n_iter=100)  # You can adjust the number of states and iterations
        my_hmm_model.fit(X_train_vec, lengths=[len(X_train_vec)])

        # Make predictions on the testing set using HMM
        _, hmm_pred = my_hmm_model.decode(X_test_vec, algorithm="viterbi")

        # Evaluate the performance of the HMM model
        hmm_accuracy = metrics.accuracy_score(y_test, hmm_pred)
        print(f"HMM Accuracy: {hmm_accuracy:.2f}")

# Example usage
my_hmm(sentence, extract_features, 'pos', word_pos=True)


MultinomialHMM has undergone major changes. The previous version was implementing a CategoricalHMM (a special case of MultinomialHMM). This new implementation follows the standard definition for a Multinomial distribution (e.g. as in https://en.wikipedia.org/wiki/Multinomial_distribution). See these issues for details:
https://github.com/hmmlearn/hmmlearn/issues/335
https://github.com/hmmlearn/hmmlearn/issues/340


HMM Accuracy: 0.00


# Question [4]: DL Classification

## [4.1]


## [4.2]

## [4.3]

The Comparison Dictionary example

In [ ]:
comparison_table = {}

comparison_table['question_step_number'] = []
comparison_table['model_name'] = []
comparison_table['features'] = []
comparison_table['accuracy'] = []

In [ ]:
import pandas as pd

df = pd.DataFrame(comparison_table)
df

In [ ]:
df.to_csv("[your_name].csv", index=False)

<h1 style="text-align: center;">Sequence to Sequence<h1>

Prepare libraries

In [ ]:
# here put every import you need e.g. import nltk
# it's better to load what you need from the package by from [] import [] instead of import the whole package
!pip install arabert

Download data

In [ ]:
# !wget 'https://drive.google.com/uc?export=download&id=1OoUGXNKnmm3KYabreBFWMV9mr2JwQOJH' -O 'test.csv'
# !wget 'https://drive.google.com/uc?export=download&id=1xnw6kKitwQwYgaOAQppDbj6T3ARCwRqz' -O 'train.csv'

In [ ]:
from arabert import ArabertPreprocessor
from arabert.aragpt2.grover.modeling_gpt2 import GPT2LMHeadModel

# [Extra 1]

# [2]

# [3]